<div class="alert alert-block alert-success">  
<h1><center><strong>Time Series Problem ⏰</strong></center></h1>

#### If this helped in your learning, then please **UPVOTE** – as they are the source of motivation!

#### **What are time series forecasting and analysis?**

Time series forecasting is a set of methods in statistics and data science to predict some variables that develop and change over time. The underlying intention of time series forecasting is determining how target variables will change in the future by observing historical data from the time perspective, defining the patterns, and yielding short or long-term predictions on how change occurs – considering the captured patterns.


Time series problems, on the other hand, are always time-dependent and we usually look at four main components: seasonality, trends, cycles, and irregular components.


![](https://content.altexsoft.com/media/2017/07/medicine-sales-graph.png.webp)



The graph above is a clear example of how trends and seasons work.

Trends. The trend component describes how the variable – drug sales in this case – changes over long periods of time. We see that the sales revenues of antidiabetic drugs have substantially increased during the period from the 1990s to 2010s.

Seasons. The seasonal component showcases each year’s wave-like changes in sales patterns. Sales were increasing and decreasing seasonally. Seasonal series can be tied to any time measurement, but these periods of time always have a fixed length and succession. We can consider monthly or quarterly patterns for sales in midsize or small eCommerce, or track micro-interactions across a day.

Cycles. Cycles are long-term patterns that have a waveform and recurring nature similar to seasonal patterns but with variable length, they don’t have a fixed time period. For example, business cycles have recognizable elements of growth, recession, and recovery. But the cycles themselves stretch in time differently for a given country throughout its history.

Irregularities. Irregular components appear due to unexpected events, like cataclysms, or are simply representative of noise in the data.

<div class="alert alert-info">  
<h3><strong>Import Libraries 📃</strong></h3>
</div>

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import matplotlib.patches as patches
# import pmdarima as pm
# from pmdarima.arima import auto_arima

In [3]:
sales_train = pd.read_csv(r'./data/train.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
items_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')


train = pd.read_csv(r'./data/train.csv') # usecols = ['']
oil = pd.read_csv(r'./data/oil.csv')
stores = pd.read_csv(r'./data/stores.csv')
transactions = pd.read_csv(r'./data/transactions.csv')


FileNotFoundError: [Errno 2] No such file or directory: '../input/competitive-data-science-predict-future-sales/sales_train.csv'

In [ ]:
sales_train.head(2)

In [ ]:
items.head(2)

In [ ]:
items_cat.head(2)

In [ ]:
shops.head(2)

**First Merge these Tables by using Join technique.**

In [ ]:
item_merged = pd.merge(items,items_cat,how='inner')
shop_merged = pd.merge(sales_train,shops,on='shop_id')
train_data = pd.merge(item_merged,shop_merged,on='item_id')

In [ ]:
train_data.head()

# Remove Duplicated Rows

In [ ]:
train_data.drop(train_data[train_data.duplicated()].index,axis=0,inplace=True)

In [ ]:
train_data.info()

There are only one Item which have the price of negative 1. it can be an outlier so we will remove it with most occurring value.

In [ ]:
train_data[train_data.item_price < 0]

In [ ]:
train_data.loc[954789,'item_price'] = train_data['item_price'].mode()[0]

**Groupby function use to group the data with the packet of date_block_num,shop_id,item_id and count the item_cnt_day to find out item count in month**

In [ ]:
temp_data = train_data.groupby(['date_block_num','shop_name','item_name','item_category_name']).agg({'item_cnt_day' : 'count'})
temp_data.reset_index(inplace=True)
temp_data = temp_data.set_index('date_block_num')

In [ ]:
temp_data.head(2)

**Checking Which item sold the most**

In [ ]:
plt.figure(figsize=(16,10))
plt.title('Shops for most item Count',weight='bold',fontsize=15)
temp_data.groupby('shop_name')['item_cnt_day'].sum().sort_values(ascending=False)[0:20].plot(kind='line',marker='*',color="#2D51A5")
temp_data.groupby('shop_name')['item_cnt_day'].sum().sort_values(ascending=False)[0:20].plot(kind='bar',color="#5EAFD9")

**Checking which shop sold the items most**

In [ ]:
plt.figure(figsize=(16,6))

temp_data.groupby('item_name')['item_cnt_day'].sum().sort_values(ascending=False)[0:20].plot(kind='line',marker='*',color='#d95f0e')
temp_data.groupby('item_name')['item_cnt_day'].sum().sort_values(ascending=False)[0:20].plot(kind='bar',color='#2c7fb8')

arrow_data = 'Simple,tail_width=2,head_width=14,head_length=16'
kw = dict(arrowstyle=arrow_data,color='#c51b8a')
arr = patches.FancyArrowPatch((4.5,23000),(1.0,20000),connectionstyle='arc3,rad=.10',**kw)
plt.gca().add_patch(arr)
plt.text(x=3.6,y=24000,s="Most Sold Item",color='black',size=17)

**Our Month Column is in form of Str. so first convert it into datetime form**

In [ ]:
train_data['date'] = pd.to_datetime(train_data['date'],format='%d.%m.%Y')
train_data['Month'] = train_data['date'].dt.to_period('M')
train_data['Month'] = train_data['Month'].astype(str)

**Now we have all the Months+year column named Month, now applying groupby to count the item_cnt_day for whole month**

In [ ]:
temp_data = train_data.groupby(['shop_id','item_id','Month']).agg({'item_cnt_day' : 'sum'})
temp_data.reset_index(inplace=True)
temp_data = temp_data.set_index('Month')

In [ ]:
temp_data.rename(columns = {'item_cnt_day' : 'item_cnt_month'},inplace=True)
temp_data.head()

**Checking which Month with year selling the Most**

In [ ]:
plt.figure(figsize=(18,10))
temp_data.groupby('Month')['item_cnt_month'].sum().sort_values(ascending=False)[:20].plot(kind='bar',color='#2b8cbe')

In [ ]:
plt.figure(figsize=(18,10))
temp_data.groupby('Month')['item_cnt_month'].sum().plot(legend=True,color='forestgreen')

**So we determine that Last Months like November, December with different years have most selling**

In [ ]:
temp_data.reset_index(inplace=True)
temp_data.tail(5)

In [ ]:
X = temp_data.iloc[:,:-1]
y = temp_data.iloc[:,-1:]
le = LabelEncoder()
X['Month']= le.fit_transform(X['Month'])
X

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
x_train.head()

**Using XGBRegressor with different hyperparameter tuning**

In [ ]:
xg = xgb.XGBRegressor(base_score=0.5,
                      n_estimators=400,
                      min_child_weight=1,
                     max_depth=7,
                     learning_rate=0.1,
                     booster='gbtree',
                     tree_method='exact',
                     reg_alpha=0,
                     subsample=0.5,
                     validate_parameters=1,
                     colsample_bylevel=1,
                     colsample_bynode=1,
                     colsample_bytree=1,
                     gamma=0)
xg.fit(x_train,y_train)
xg.score(x_test,y_test)

In [ ]:
x_train

In [26]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
test.set_index('ID',inplace=True)
test['Month'] = 34

In [27]:
pred = xg.predict(test)
test['item_cnt_month'] = (pred[0]*len(test))/len(test)

In [28]:
sub = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
test.reset_index(inplace=True)
sub = test.drop(['shop_id','item_id','Month'],axis=1)

In [29]:
sub.to_csv('submission.csv',index=False)
subb = pd.read_csv('./submission.csv')
subb.head()

,ID,item_cnt_month
0,0,0.972291
1,1,0.972291
2,2,0.972291
3,3,0.972291
4,4,0.972291
